# **Recommendation Agent**

The **Recommendation Agent** is the final agent in the recommendation process. It orchestrates how the **Activity Agent**, **Usage Agent**, **Load Agent** and **Price Agent** interact in order to deliver a recommendation at a particular date for a given household (see the dedicated notebooks in order to see the definition of these agents in detail).

The recommendation agent works as follows:

1. It requests the outputs of the **Activity Agent**, **Usage Agent**, **Load Agent** and **Price Agent**:


*   The Activity Agent returns the probability that persons are present and in an "active state" in the house at each given hour of the day.
*   The Usage Agent returns the probability that a given to-be-recommended-device will be used on the next day.
*   The Load Agent returns a typical load profile for each to-be-recommended-device.
*   The Price Agent returns the day-ahead-prices for the next 48 hours.

2. It then computes the cost associated with launching the devices at each hour of the next day (based on the devices' typical load profiles and the day ahead electricity prices).

3. Finally it recommends the cheapest launching hour among the set of hours at which users are likely present and active **[Probability(Present&Active)> Threshold]**. A recommendation  for a given device is made only if the user is likely enough to use the device on the next day **[Probability(Device_Usage)> Threshold]**


In the present notebook, we will build this **Recommendation Agent** step by step. For that purpose, we will first load and preprocess the required data with the preprocessing agents. Then, we will iteratively add functions to the **Recommendation Agent class** in order to finally build a function entitled "Pipeline", which ouputs the desired recommendations.

## **1. Load and Pre-process Data**

This part's only purpose is to load the data used in the recommendation agent. This process is described in detail in the Preparation Agent.  **[You might need to adapt some parameters when applying the script to another household than household 1]**

### **1.1 Initialize and load python scripts**

In [ ]:
# loading necessary libraries
#import knn as knn
import pandas as pd
import numpy as np
import os

import time
import shap as shap
import pickle



from helper_functions import Helper
from agents import Preparation_Agent

# Gammli
from sklearn.model_selection import train_test_split
from collections import OrderedDict
from sklearn.metrics import mean_squared_error,roc_auc_score,mean_absolute_error,log_loss

from gammli.gammli import GAMMLI
from gammli.dataReader import data_initialize
from gammli.utils import local_visualize
from gammli.utils import global_visualize_density
from gammli.utils import feature_importance_visualize
from gammli.utils import plot_trajectory
from gammli.utils import plot_regularization
import tensorflow as tf


helper = Helper()

In [ ]:
# ML Models

# More ML Models
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB


In [ ]:
DATA_PATH = '../data/'

### **1.2 Set Params**

**Note:** For the full detail of the parameters and the preprocessing agents take a look at the Preparation Agent's notebook.

In [ ]:
# load household data for Household 1
household = helper.load_household(DATA_PATH, 1)

In [ ]:
threshold = 0.01
active_appliances = ['Toaster', 'Tumble Dryer', 'Dishwasher', 'Washing Machine','Television', 'Microwave', 'Kettle']
shiftable_devices = ['Tumble Dryer', 'Washing Machine', 'Dishwasher']
#model_types = ['logit', 'knn', 'ada', 'random forest']

In [ ]:
#activity params
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 0
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}

activity_params = {
    'active_appliances': active_appliances,
    'threshold': threshold 
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

#load agent
device_params = {
    'threshold': threshold
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

#usage agent

device = {
    'threshold' : threshold}

aggregate_params24_H = {
    'resample_param': '24H'
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}

In [ ]:
#calling the preparation pipeline
#prep = Preparation_Agent(household)
#activity_df = prep.pipeline_activity(household, activity_pipe_params)
#load_df, _, _ = prep.pipeline_load(household, load_pipe_params)
#usage_df = prep.pipeline_usage(household, usage_pipe_params)

#load price data
#FILE_PATH = '/content/drive/MyDrive/T4_Recommendation-system-for-demand-response-and-load-shifting/02_data/'
#price_df = helper.create_day_ahead_prices_df(DATA_PATH, 'Day-ahead Prices_201501010000-201601010000.csv')


In [ ]:
#activity_df.to_pickle('../data/processed_pickle/activity_df.pkl')
#load_df,_, _ .to_pickle('../data/processed_pickle/load_df.pkl')
#usage_df.to_pickle('../data/processed_pickle/usage_df.pkl')
#price_df.to_pickle('../data/processed_pickle/price_df.pkl')

In [ ]:
# Load pickle data
activity_df = pd.read_pickle('../data/processed_pickle/activity_df.pkl')
load_df = pd.read_pickle('../data/processed_pickle/load_df.pkl')
usage_df = pd.read_pickle('../data/processed_pickle/usage_df.pkl')
price_df = pd.read_pickle('../data/processed_pickle/price_df.pkl')

## **2. Constructing the Recommendation Agent**

### **2.1 Initiliaze Agent**
In a first step, the Recommendation Agent is initialized with the preprocessed data and the name of the shiftable devices (those for which we want to make predictions). All Agents are initialized accordingly.

In [ ]:
from agents import Activity_Agent, Usage_Agent, Load_Agent, Price_Agent
class Recommendation_Agent:
    import pandas as pd

    def __init__(self, activity_input, usage_input, load_input, price_input, shiftable_devices, model_type):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices
        self.model_type = model_type

        self.Activity_Agent = Activity_Agent(activity_input)

        #create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {name: Usage_Agent(usage_input , name)  for name in shiftable_devices}

        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent(price_input)

In [ ]:
#initialize the Recommendation Agent with the required inputs
recommend = Recommendation_Agent(activity_df, usage_df, load_df, price_df, shiftable_devices,model_type='knn')

### **2.2 Compute Usage Cost For Every Starting Time (each of the 24 hours of the day) For A Given Device**

This function computes the cost associated with launching a to-be-recommended-device at each hour of the next day. 

#### **2.2.1 Electricity Prices For 24 hours After Hypothetical Starting Time**
First we build up a function which gives the electricity price for the 24 hours following a hypothetical starting time. 

*   The column "Price_at_H+0" gives the electricity price for the 24 hours after 00:00:00
*   The column "Price_at_H+1" gives the electricity price for the 24 hours following 01:00:00 ( "Price_at_H+0" shifted by one hour)
* The column "Price_at_H+2" gives the electricity price for the 24 hours following 02:00:00 ( "Price_at_H+0" shifted by two hours)
*....

This function first requests the **day-ahead electricity prices** for the next 48h from the Price_Agent. Then it arranges the prices as described above.


In [ ]:
def electricity_prices_from_start_time(self, date):
  import pandas as pd
  prices_48 = self.Price_Agent.return_day_ahead_prices(date)
  prices_from_start_time = pd.DataFrame()

  for i in range(24):
    prices_from_start_time["Price_at_H+"+ str(i)] = prices_48.shift(-i)

  #delete last 24 hours
  prices_from_start_time = prices_from_start_time[:-24]
  return prices_from_start_time

# add to Activity agent
setattr(Recommendation_Agent, 'electricity_prices_from_start_time', electricity_prices_from_start_time)
del electricity_prices_from_start_time

In [ ]:
recommend.electricity_prices_from_start_time("2014-02-20")
#H0 prices for next 24 hours start at 00:00
#H1 prices for next 24 hours start at 01:00

#### **2.2.2 Device Launching Cost By Hour Of The Day**
We compute the cost of operating the device at every given hour by multiplying the **day ahead electricity price** with the device's **typical load profile**. The latter typical load profile is generated by the **Load Agent** (for details see the Load Agent's notebook).

As an output, we get the typical costs of operating the device for all possible 24 staring times.

**Note:** The Recommendation Agent's pipeline functions can be sped up by providing the predecessing agents' outputs directly. This Functionality will be used to evaluate the recommender systems performance.

In [ ]:
def cost_by_starting_time(self, date, device, evaluation=False):
    import numpy as np
    import pandas as pd

    # get electriciy prices following every device starting hour with previously defined function
    prices = self.electricity_prices_from_start_time(date)

    # build up table with typical load profile repeated for every hour (see Load_Agent)
    if not evaluation:
        device_load = self.Load_Agent.pipeline(self.load_input, date, self.shiftable_devices).loc[device]
    else:
        # get device load for one date
        device_load = evaluation["load"][date].loc[device]

    device_load = pd.concat([device_load] * 24, axis=1)

    # multiply both tables and aggregate costs for each starting hour
    costs = np.array(prices) * np.array(device_load)
    costs = np.sum(costs, axis=0)

    # return an array of size 24 containing the total cost at each staring hour.
    return costs

setattr(Recommendation_Agent, 'cost_by_starting_time', cost_by_starting_time)
del cost_by_starting_time

As can be seen below, the output returns the cost associated with starting the "Washing machine" on each hour of the "2014-02-20".

In [ ]:
recommend.cost_by_starting_time("2014-02-20", "Washing Machine")

### **2.3 Starting Time Recommendation For Each Device**

We create a function that gives a starting time recommendation for a given device.

1.   The function loads the **device usage costs associated with each starting time** (see above function).
2.   In order **not** to recommend to start the device at an hour where the person is not home or at sleep, we exclude hours that have an  **activity probability below a certain threshold**. These probabilities are computed with the **Activity Agent**.
3. In order **not** to make a recommendation when the household is unlikely to use the device on the next day anyway, we set a **device usage probability threshold** under which no recommendation is made. These usage probabilities are computed with the **Usage Agent**.


The function outputs a dictionary with the best starting time, among the hours at which activity is likely enough. The output additionaly contains **"no_recommend"** flags, in order to signal that no recommendation should be made when :
* There is no hour of the day where activity is likely enough (all hours of the day have an activity probability below the set threshold), then the **"no_recommend_flag_activity"** turns from 0 to 1.
* The device usage is unlikely (below the set threshold), then the **"no_recommend_flag_usage"** turns from 0 to 1.


In [ ]:
#return cheapest launching hour from the set of hours satifsfying:  probability of activity > threshold
def recommend_by_device(self, date, device, model_type, activity_prob_threshold, usage_prob_threshold, evaluation=False):
    import numpy as np

    split_params = {
        "train_start": "2013-11-01",
        "test_delta": {"days": 1, "seconds": -1},
        "target": "activity",
    }

    # compute costs by launching time:
    costs = self.cost_by_starting_time(date, device, evaluation=evaluation)

    # compute activity probabilities
    if not evaluation:
        activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, model_type ,split_params)
    else:
        # get activity probs for date
        activity_probs = evaluation["activity"][date]

    # set values above threshold to 1. Values below to Inf
    # (vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
    activity_probs = np.where(activity_probs >= activity_prob_threshold, 1, float("Inf"))

    # add a flag in case all hours have likelihood smaller than threshold
    no_recommend_flag_activity = 0
    if np.min(activity_probs) == float("Inf"):
        no_recommend_flag_activity = 1

    # compute cheapest hour from likely ones
    best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

    # compute likelihood of usage:
    if not evaluation:
        usage_prob = self.Usage_Agent[device].pipeline(self.usage_input, date, model_type, split_params["train_start"])
    else:
        # get usage probs
        name = "usage_" + device.replace(" ", "_").replace("(", "").replace(")", "").lower()
        usage_prob = evaluation[name][date]

    no_recommend_flag_usage = 0
    if usage_prob < usage_prob_threshold:
        no_recommend_flag_usage = 1

    return {
        "recommendation_date": [date],
        "device": [device],
        "best_launch_hour": [best_hour],
        "no_recommend_flag_activity": [no_recommend_flag_activity],
        "no_recommend_flag_usage": [no_recommend_flag_usage],
        "recommendation": [best_hour if (no_recommend_flag_activity == 0 and no_recommend_flag_usage == 0)else np.nan]
    }

setattr(Recommendation_Agent, 'recommend_by_device', recommend_by_device)
del recommend_by_device

In [ ]:
recommend.recommend_by_device("2014-08-21", "Dishwasher", 'knn', 0.3, 0.3)

### **2.4 Create Recommendation Function For Entire Household**
Finally, we wrap up the "recommend_by_device" function that makes a recommendation for each device, into the "pipeline" function that will make recommendations for all shiftable devices within a household.

In [ ]:
def pipeline(self, date, model_type, activity_prob_threshold, usage_prob_threshold, evaluation=False):
    import pandas as pd

    recommendations_by_device = self.recommend_by_device(date, self.shiftable_devices[0], model_type, activity_prob_threshold,              usage_prob_threshold, evaluation=evaluation)
    recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

    for device in self.shiftable_devices[1:]:
        recommendations_by_device = self.recommend_by_device(date, device, model_type, activity_prob_threshold, usage_prob_threshold, evaluation=evaluation)
        recommendations_table = recommendations_table.append(pd.DataFrame.from_dict(recommendations_by_device))
    return recommendations_table

setattr(Recommendation_Agent, 'pipeline', pipeline)
del pipeline

We can then generate a recommendation for a household by specifying:
1. The day to be recommended
2. The "activity_prob_threshold" (hours that have a smaller probability of household activity are not considered for recommendation)
3. The "usage_probability_threshold" (devices that have a smaller probability of usage are not considered for recommendation)

**Note**:  It remains to be investigated at which value to set these thresholds.

In [ ]:
recommend.pipeline(date = "2015-02-15", model_type= "random forest", activity_prob_threshold = 0.4,  usage_prob_threshold = 0.3)

Finally, the system recommends to the user the "best_launch_hour" on the "recommendation_date" if both the "no_recommend_flag_activity" and "no_recommend_flag_usage" are at 0 for the given device. 

In [ ]:
#### Continue here:
# TO DO: get out specific models from rec agents; already extraction possible?
#

#TO DO: more than just model_type as setting?
# e.g. pickle final models & hyperparameters from file?

#TO DO: streamline s.t. recommendation pipeline is only called with one call

#What do we want to see at the end?
#overview of different explainers & their performance on metrics
#we need: function that implements explainer on individual agent
# for local prediction from recommendation


In [ ]:
# Examplarily
#Activity Agent  with shap
# what we need to get out model, Xtest,location of rec?
# problem:


import shap
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10)
shap.initjs()

In [ ]:
date = '2014-01-01'
activity = Activity_Agent(activity_df)
X_test = activity.get_Xtest(activity_df, date)
X_test


#date = '2014-01-01'
#activity = Activity_Agent(activity_df)
y_test = activity.get_ytest(activity_df, date)
y_test

In [ ]:
#date = '2014-01-01'

activity = Activity_Agent(activity_df)
X_train = activity.get_Xtrain(activity_df, date)
X_train

activity = Activity_Agent(activity_df)
y_train = activity.get_ytrain(activity_df, date)
y_train

In [ ]:
X_train, y_train, X_test, y_test = activity.train_test_split(activity_df, date)
activity_df.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape


## Explainability Evaluation BOARD

## Explainability of Activity Agent

In [ ]:
#import sys
#!{sys.executable} -m pip install shap

In [ ]:
pred_model_knn = activity.fit_knn(X_train,y_train)
pred_model_rf = activity.fit_random_forest(X_train,y_train)
pred_model_ada = activity.fit_ADA(X_train,y_train)

In [ ]:
X_test.iloc[0]

In [ ]:
X_test_i = np.array(X_test.iloc[0]).reshape(1,-1)
X_test_i

In [ ]:
y_pred_knn = pred_model_knn.predict(X_test_i)
y_pred_knn

In [ ]:
y_test

In [ ]:
# that means we can get an error estimate for the knn model. how is that different with the shap model?

In [ ]:
predictive_models = [pred_model_knn, pred_model_rf, pred_model_ada]
predictive_models
#y_test.head(20)#.iloc[2]

In [ ]:
### LIME Function
from lime import lime_tabular
import statistics

local = 9 #the instance we want to explain
n_iter = 1
i = 0

data = {'Explainability Model': [],
        'Predictive Model': [],
        'Classifier': [],
        'Run Duration': [],
        'MAEE': [],
        'MSEE': []}
exp_eval_df = pd.DataFrame(data)

for pred_model in predictive_models:
    classifier = pred_model
    print(classifier)
    
    print(pred_model)
    print(str(pred_model))
    
    if "KNeighbors" in str(pred_model):
        predictive_model = "KNN"
    elif "Random" in str(pred_model):
        predictive_model = "Random Forest"
    elif "Ada" in str(pred_model):
        predictive_model = "AdaBoost"
    else:
        predictive_model = "Unknown model"
        
    print(predictive_model)
    
    #predicted activity by pred model
    X_test_i = np.array(X_test.iloc[local]).reshape(1,-1)
    y_pred_i = pred_model.predict(X_test_i)
    print(y_pred_i[0])

    # LIME
    explainability_model = 'LIME'
    start_time = time.time()
    explainer = lime_tabular.LimeTabularExplainer(training_data = np.array(X_train),
                                              mode = "regression",
                                              feature_names = X_train.columns,
                                              categorical_features = [0])

    exp = explainer.explain_instance(data_row = X_test.iloc[local], #changed to somewhere where activity= 1
                                predict_fn = pred_model.predict)

    exp.show_in_notebook(show_table = True)
    end_time = time.time()
    difference_time = end_time - start_time
        
    #compute MSEE:
    y_expl_i = exp.local_pred
    #print(y_expl_i)
    #SEE = (y_pred_i - y_expl_i)**2 #squared prediction error for this computation (repetition necessary for MSEE)
    
    rep = 0

    exp_list_abs = []
    exp_list_squ = []
    for rep in range(n_iter): #number of iterations for computing the diffferent lime models 
 
        exp = explainer.explain_instance(data_row = X_test.iloc[local], #changed to somewhere where activity= 1 
                                predict_fn = pred_model.predict)
        exp_list_abs.append(y_pred_i-exp.local_pred)
        exp_list_squ.append((y_pred_i-exp.local_pred)**2)

    exp_np_abs =np.array(exp_list_abs)
    exp_np_squ =np.array(exp_list_squ)
    #exp_list_abs.to_numpy()
    #exp_list_abs.flatten()
    #print(exp_list_abs.flatten())
           
    MAEE = statistics.mean(exp_np_abs.flatten())
    MSEE = statistics.mean(exp_np_squ.flatten())
    print(MAEE)
    
    #MSEE = mean(exp_list_squ[0])
    #print(MSEE)
    
    #exp.as_list()
    exp_eval_df.loc[i+1] = [explainability_model, predictive_model, classifier, difference_time, MAEE, MSEE]
    
    i=i+1
    
    
    # SHAP

    explainability_model = 'SHAP'
    start_time = time.time()

    explainer = shap.KernelExplainer(pred_model.predict_proba, X_train)
    shap_values = explainer.shap_values(X_test.iloc[local,:])
    display(shap.force_plot(explainer.expected_value[1], shap_values[1], X_test.iloc[local,:]))

    end_time = time.time()
    difference_time = end_time - start_time
    
    #compute MSEE:
    rep = 0
    
    shap_list_abs = []
    shap_list_squ = []
    
    for rep in range(n_iter):
        explainer = shap.KernelExplainer(pred_model.predict_proba, X_train)
        shap_values = explainer.shap_values(X_test.iloc[local,:])
        print(shap_values)
        # first array = contribution to class 0
        # second array = contribution to class 1
        contribution_to_class_1 = np.array(shap_values).sum(axis=1)[1] # the red part of the diagram
        print(contribution_to_class_1)
        base_value = explainer.expected_value[1] # the mean prediction
        print(base_value)
        y_expl_i = base_value + contribution_to_class_1
        print(y_expl_i)
        SEE = (y_pred_i[0] - y_expl_i)**2 #squared prediction error for this computation (repetition necessary for MSEE)
    
        shap_list_abs.append(y_pred_i-y_expl_i)
        shap_list_squ.append((y_pred_i-y_expl_i)**2)

        print(shap_list_abs)
    
    shap_np_abs =np.array(shap_list_abs)
    shap_np_squ =np.array(shap_list_squ)

    MAEE = statistics.mean(shap_np_abs.flatten())
    MSEE = statistics.mean(shap_np_squ.flatten())
    print(MAEE)

    exp_eval_df.loc[i+1] = [explainability_model, predictive_model, classifier, difference_time, MAEE, MSEE]

    i = i+1
    
    print(exp_eval_df)

In [ ]:
exp_eval_df

## Explainable Booster Classifier 


In [2]:
from interpret.glassbox import ExplainableBoostingClassifier

ebm = ExplainableBoostingClassifier()
#ebm.fit(X_train, y_train)


In [3]:
type(ebm)

interpret.glassbox.ebm.ebm.ExplainableBoostingClassifier

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
from interpret import show

ebm_global = ebm.explain_global()
show(ebm_global)

In [ ]:
ebm_local = ebm.explain_local(X_test, y_test)
show(ebm_local)

## Gammli -  Generalized Additive Modeling with Manifest and Latent Interactions

In [ ]:
X_train.info()

In [ ]:
columns = X_train.columns

In [ ]:
columns

In [ ]:
random_state = 0
task_type = "Classification"

meta_info = OrderedDict()

for i in columns:
    meta_info[i]={'type': 'continues','source':'user'}
meta_info['hour']={'type': 'continues','source':'user'}
meta_info['activity_lag_24']={'type': 'continues','source':'item'}
meta_info['activity_lag_48']={'type': 'continues','source':'item'}
meta_info['day_name_Monday']={'type': 'continues','source':'item'}
meta_info['day_name_Saturday']={'type': 'continues','source':'item'}
meta_info['day_name_Sunday']={'type': 'continues','source':'item'}
meta_info['day_name_Thursday']={'type': 'continues','source':'item'}
meta_info['day_name_Wednesday']={'type': 'continues','source':'item'}

In [ ]:
# run Gammli
tr_x, tr_Xi, tr_y, tr_idx, te_x, te_Xi, te_y, val_x, val_Xi, val_y, val_idx, meta_info, model_info,sy,sy_t = data_initialize(X_train,X_test,meta_info,task_type ,'warm', random_state, True)
model = GAMMLI(wc='warm',model_info=model_info, meta_info=meta_info, subnet_arch=[20, 10],interact_arch=[20, 10],activation_func=tf.tanh, batch_size=min(500, int(0.2*tr_x.shape[0])), lr_bp=0.001, auto_tune=False,
               interaction_epochs=1000,main_effect_epochs=1000,tuning_epochs=200,loss_threshold_main=0.01,loss_threshold_inter=0.1,
              verbose=True, early_stop_thres=20,interact_num=10,n_power_iterations=5,n_oversamples=10, u_group_num=10, i_group_num=10, reg_clarity=10, lambda_=5,
              mf_training_iters=200,change_mode=False,convergence_threshold=0.0001,max_rank=3,interaction_restrict='intra', si_approach ='als')
model.fit(tr_x, val_x, tr_y, val_y, tr_Xi, val_Xi, tr_idx, val_idx)

## Explainability of Usage Agent

In [ ]:
usage_df

In [ ]:
date = '2014-11-01' #other date than activity
Usage_Agent_i = Usage_Agent(usage_df, "Dishwasher")

In [ ]:
X_train, y_train, X_test, y_test = Usage_Agent_i.train_test_split(usage_df, "2014-11-01", train_start='2013-11-01')

In [ ]:
X_train.shape

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test.shape

In [ ]:
usage = Usage_Agent(usage_df, "Dishwasher")
model = usage.fit_knn(X_train, np.ravel(y_train)) # Change it to the model you want to use
print(model)

In [ ]:
pred_model_knn = usage.fit_knn(X_train,y_train)
pred_model_rf = usage.fit_random_forest(X_train,y_train)
pred_model_rf

In [ ]:
pred_model_knn = usage.fit_knn(X_train,np.ravel(y_train))
pred_model_rf = usage.fit_random_forest(X_train,np.ravel(y_train))
pred_model_rf

In [ ]:
predictive_models = [pred_model_knn, pred_model_rf]
y_test.iloc[0]

In [ ]:
#compute prediction at day t (see date used for split sampling)
import numpy as np
y_hat = usage.predict(model, X_test)
y_hat

In [ ]:
X_test#.iloc[2]#

In [ ]:
### LIME Function
from lime import lime_tabular

i = 0

for pred_model in predictive_models:
    predictive_model = pred_model

    # LIME
    explainability_model = 'LIME'
    start_time = time.time()
    explainer = lime_tabular.LimeTabularExplainer(training_data = np.array(X_train),
                                              mode = "regression",
                                              feature_names = X_train.columns,
                                              categorical_features = [0])

    exp = explainer.explain_instance(data_row = X_test, #removed iloc 
                                predict_fn = pred_model.predict)

    exp.show_in_notebook(show_table = True)
    end_time = time.time()
    difference_time = end_time - start_time
    #exp.as_list()
    exp_eval_df.loc[i+1] = [explainability_model, predictive_model, start_time, end_time, difference_time]

    # SHAP

    explainability_model = 'SHAP'
    start_time = time.time()

    explainer = shap.KernelExplainer(pred_model.predict_proba, X_train)
    print(explainer)
    shap_values = explainer.shap_values(X_test)#.iloc[0,:]) #removed iloc
    print(shap_values)
    display(shap.force_plot(explainer.expected_value[1], shap_values[1], X_test))#removed iloc, display necessary to show multiple
    
    end_time = time.time()
    difference_time = end_time - start_time

    exp_eval_df.loc[i+2] = [explainability_model, predictive_model, start_time, end_time, difference_time]

    i = i+2

In [ ]:
exp_eval_df

## Explainale Boosting Classifier - Usage

In [ ]:
ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)


In [ ]:
ebm_global = ebm.explain_global()
show(ebm_global)

In [ ]:
#ebm_local = ebm.explain_local(X_test, y_test)
#show(ebm_local)

## **Appendix A1: Complete Recommendation Agent Class**

In [ ]:
class Recommendation_Agent:
    def __init__(
        self, activity_input, usage_input, load_input, price_input, shiftable_devices
    ):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices
        self.Activity_Agent = Activity_Agent(activity_input)
        # create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {
            name: Usage_Agent(usage_input, name) for name in shiftable_devices
        }
        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent(price_input)

    # calculating costs
    # -------------------------------------------------------------------------------------------
    def electricity_prices_from_start_time(self, date):
        import pandas as pd

        prices_48 = self.Price_Agent.return_day_ahead_prices(date)
        prices_from_start_time = pd.DataFrame()
        for i in range(24):
            prices_from_start_time["Price_at_H+" + str(i)] = prices_48.shift(-i)
        # delete last 24 hours
        prices_from_start_time = prices_from_start_time[:-24]
        return prices_from_start_time

    def cost_by_starting_time(self, date, device, evaluation=False):
        import numpy as np
        import pandas as pd

        # get electriciy prices following every device starting hour with previously defined function
        prices = self.electricity_prices_from_start_time(date)
        # build up table with typical load profile repeated for every hour (see Load_Agent)
        if not evaluation:
            device_load = self.Load_Agent.pipeline(
                self.load_input, date, self.shiftable_devices
            ).loc[device]
        else:
            # get device load for one date
            device_load = evaluation["load"][date].loc[device]
        device_load = pd.concat([device_load] * 24, axis=1)
        # multiply both tables and aggregate costs for each starting hour
        costs = np.array(prices) * np.array(device_load)
        costs = np.sum(costs, axis=0)
        # return an array of size 24 containing the total cost at each staring hour.
        return costs
    
    # creating recommendations
    # -------------------------------------------------------------------------------------------
    def recommend_by_device(
        self,
        date,
        device,
        model_type,
        activity_prob_threshold,
        usage_prob_threshold,
        evaluation=False,
    ):
        import numpy as np

        # add split params as input
        # IN PARTICULAR --> Specify date to start training
        split_params = {
            "train_start": "2013-11-01",
            "test_delta": {"days": 1, "seconds": -1},
            "target": "activity",
        }
        # compute costs by launching time:
        costs = self.cost_by_starting_time(date, device, evaluation=evaluation)
        # compute activity probabilities
        if not evaluation:
            activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, model_type, split_params)
        else:
            # get activity probs for date
            activity_probs = evaluation["activity"][date]

        # set values above threshold to 1. Values below to Inf
        # (vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
        activity_probs = np.where(activity_probs >= activity_prob_threshold, 1, float("Inf"))

        # add a flag in case all hours have likelihood smaller than threshold
        no_recommend_flag_activity = 0
        if np.min(activity_probs) == float("Inf"):
            no_recommend_flag_activity = 1

        # compute cheapest hour from likely ones
        best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

        # compute likelihood of usage:
        if not evaluation:
            usage_prob = self.Usage_Agent[device].pipeline(self.usage_input, date, model_type, split_params["train_start"])
        else:
            # get usage probs
            name = ("usage_"+ device.replace(" ", "_").replace("(", "").replace(")", "").lower())
            usage_prob = evaluation[name][date]

        no_recommend_flag_usage = 0
        if usage_prob < usage_prob_threshold:
            no_recommend_flag_usage = 1

        return {
            "recommendation_date": [date],
            "device": [device],
            "best_launch_hour": [best_hour],
            "no_recommend_flag_activity": [no_recommend_flag_activity],
            "no_recommend_flag_usage": [no_recommend_flag_usage],
            "recommendation": [
                best_hour
                if (no_recommend_flag_activity == 0 and no_recommend_flag_usage == 0)
                else np.nan
            ],
        }

    # vizualizing the recommendations
    # -------------------------------------------------------------------------------------------
    def recommendations_on_date_range(
        self, date_range, activity_prob_threshold=0.6, usage_prob_threshold=0.5
    ):
        import pandas as pd

        recommendations = []
        for date in date_range:
            recommendations.append(self.pipeline(date, activity_prob_threshold, usage_prob_threshold))
            output = pd.concat(recommendations)
        return output

    def visualize_recommendations_on_date_range(self, recs):
        import plotly.express as px
        import plotly.graph_objects as go

        fig = go.Figure()

        for device in recs["device"].unique():
            plot_device = recs[recs["device"] == device]
            fig.add_trace(
                go.Scatter(
                    x=plot_device["recommendation_date"],
                    y=plot_device["recommendation"],
                    mode="lines",
                    name=device,
                )
            )
        fig.show()

    def histogram_recommendation_hour(self, recs):
        import seaborn as sns

        ax = sns.displot(recs, x="recommendation", binwidth=1)
        ax.set(xlabel="Hour of Recommendation", ylabel="counts")
    
    # pipeline function: create recommendations
    # -------------------------------------------------------------------------------------------
    def pipeline(self, date, activity_prob_threshold, usage_prob_threshold, evaluation=False):
        import pandas as pd

        recommendations_by_device = self.recommend_by_device(
            date,
            self.shiftable_devices[0],
            activity_prob_threshold,
            usage_prob_threshold,
            evaluation=evaluation,
        )
        recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

        for device in self.shiftable_devices[1:]:
            recommendations_by_device = self.recommend_by_device(
                date,
                device,
                activity_prob_threshold,
                usage_prob_threshold,
                evaluation=evaluation,
            )
            recommendations_table = recommendations_table.append(
                pd.DataFrame.from_dict(recommendations_by_device)
            )
        return recommendations_table